In [1]:
import numpy as np
import nibabel as nb
import os
from nibabel.testing import data_path

def mcflirtgood(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    identity = np.diag([1,1,1,1])
    brains = np.ones((70, 70, 50, 30), dtype=np.int16)
    for i in range(30):
        d = np.random.randint(low=1, high=10, size=1, dtype=np.int16)[0]
        d2 = 20 - d
        d3 = np.random.randint(low=1, high=10, size=1, dtype=np.int16)[0]
        d4 = 20 - d3
        brain = np.ones((70,70,50), dtype=np.int16)
        zero1 = np.zeros((d,70,50), dtype=np.int16)
        zero2 = np.zeros((d2,70,50), dtype=np.int16)
        zero3 = np.zeros((70,d3,50), dtype=np.int16)
        zero4 = np.zeros((70,d4,50), dtype=np.int16)
        brain[0:d,:,:] = zero1
        brain[-d2:,:,:] = zero2
        brain[:,0:d3,:] = zero3
        brain[:,-d4:,:] = zero4
        brains[:,:,:,i] = brain
    brainimg = nb.Nifti1Image(brains, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

def mcflirtbad(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    identity = np.diag([1,1,1,1])
    brains = np.ones((70, 70, 50, 30), dtype=np.int16)
    for i in range(30):
        brain = np.ones((70,70,50), dtype=np.int16)
        n = 70
        a = np.random.randint(low=20, high=40, size=1, dtype=np.int16)[0]
        b = np.random.randint(low=20, high=40, size=1, dtype=np.int16)[0]
        c = 25
        r = np.random.randint(low=20, high=40, size=1, dtype=np.int16)[0]
        y,x,z = np.ogrid[-a:n-a,-b:n-b,-c:50-c]
        mask = x*x + y*y + z*z <= r*r
        brain[mask] = 0
        brains[:,:,:,i] = brain
    brainimg = nb.Nifti1Image(brains, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

The code above produces good and bad simulations for MCFLIRT. 

Each good simulation is a timeseries with 30 time-steps where the data is a square matrix. In each time-step, the square is moved/translated to a different location.

Each bad simulation is a timeseries with 30 time-steps where the data is a square matrix of 1s with a circular matrix of zeros masked away. In each time-step, the circle is scaled and moved differently. This simulates the motion of an amoeba-like object.

Below are screenshotted images of 1 good simulation and 1 bad simulation. Each image shows the data at 1 time-step. Clearly, only a small subset of the data is shown here. To see the entire sim data, refer to the accompanying zip files.

Good sim:

![goodsim](mcflirt_pics/goodsim0.jpg)![goodsim](mcflirt_pics/goodsim1.jpg)![goodsim](mcflirt_pics/goodsim2.jpg)![goodsim](mcflirt_pics/goodsim3.jpg)

Bad sim:

![badsim](mcflirt_pics/badsim0.jpg)![badsim](mcflirt_pics/badsim1.jpg)![badsim](mcflirt_pics/badsim2.jpg)![badsim](mcflirt_pics/badsim3.jpg)

The simulated data above looks correct.

#### Pseudocode

MCFLIRT is a motion correction algorithm for brain images. The input is a timeseries where each time-step is a brain scan. The goal is to eliminate small changes between timesteps that are caused by unwanted motion. The algorithm works as follows:

1) Pick reference scan (we set this to the first time-step)

2) For each timestep 't':

       register brain scan at time 't' to the reference scan, using FLIRT
       
That is it. As you can see, the algorithm is very simple, and the most important bit is basically just FLIRT, for which I have done a separate markdown.

The algorithm should perform very well on the good simulations. This is because the timesteps in the good simulations are simple linear translations of the reference scan (timestep 0), and so FLIRT should be effective. Almost all the motion should be corrected easily.

The algorithm should perform poorly on the bad simulations. This is because the timesteps in the bad simulations are not linear transformations of the reference scan (timestep 0), and so FLIRT should be ineffective. Almost none of the motion should be corrected.

Below, we show the results of running the algorithm on the simulated data. Again, we only show a tiny subsample of the results in order to preserve space. Please refer to the attached zip for all the data.

Good sim:

![goodsimflirted](mcflirt_pics/goodsimflirted0.jpg)![goodsimflirted](mcflirt_pics/goodsimflirted1.jpg)![goodsimflirted](mcflirt_pics/goodsimflirted2.jpg)![goodsimflirted](mcflirt_pics/goodsimflirted3.jpg)

Bad sim:

![badsimflirted](mcflirt_pics/badsimflirted0.jpg)![badsimflirted](mcflirt_pics/badsimflirted1.jpg)![badsimflirted](mcflirt_pics/badsimflirted2.jpg)![badsimflirted](mcflirt_pics/badsimflirted3.jpg)

To quantitatively assess the results, we will compute an error metric as follows: Given a simulation, for each timestep 't', we will take the difference between the scan at 't' and the reference scan (timestep 0) and sum all the elements in the matrix. This gives us the number of pixels that were "wrong" in timestep 't'. We will then average these errors for each timestep to get the error for the entire simulation. We will then divide this value by the total number of pixels in a scan (70 x 70 = 4900) and multiply by 100 to obtain the error in percentage form. 

In [ ]:
import numpy as np
import nibabel as nb
import os
from nibabel.testing import data_path

if __name__ == '__main__':
    gooderrors = []
    for i in range(10):
        flirted = nb.load(str(i) + "/flirted.nii.gz").get_data()
        toterror = 0
        refscan = flirted[:,:,25,0]
        for t in range(30):
            scan = flirted[:,:,25,t]
            scanerror = np.sum(np.absolute(refscan - scan))
            toterror = toterror + scanerror
        error = toterror / 30
        gooderrors.append(error)

    
    baderrors = []
    for i in range(10,20):
        flirted = nb.load(str(i) + "/flirted.nii.gz").get_data()
        toterror = 0
        refscan = flirted[:,:,25,0]
        for t in range(30):
            scan = flirted[:,:,25,t]
            scanerror = np.sum(np.absolute(refscan - scan))
            toterror = toterror + scanerror
        error = toterror / 30
        baderrors.append(error)


    print gooderrors
    print baderrors

[15.061224489795919, 6.8163265306122449, 36.265306122448983, 11.979591836734693, 11.857142857142858, 2.9591836734693877, 6.0, 3.489795918367347, 12.122448979591836, 7.8571428571428568]
[23.326530612244898, 27.183673469387752, 23.0, 37.285714285714285, 32.714285714285715, 24.612244897959183, 19.979591836734695, 22.285714285714285, 23.959183673469386, 22.469387755102041]

The above data will be visualized using scatter plots. We will also compute population performance by simply average the error for each simulation in the good and bad setting (separately). This will give us an average error for each setting, which we can compare.

In [ ]:
x = range(len(gooderrors))
plt.scatter(x, gooderrors, c='b', s=40, label='good sims')
x = range(len(baderrors))
plt.scatter(x, baderrors, c='r', s=40, label='bad sims')
plt.legend(loc='upper right')
plt.title('Errors')
plt.savefig("errors.png")
plt.cla()

goodavg = np.mean(gooderrors)
badavg = np.mean(baderrors)

print "Good simulations error: ", goodavg
print "Bad simulations error: ", badavg

![errors](mcflirt_pics/errors.jpg)

Good simulations error:  8.36257382247  
Bad simulations error:  25.6816326531

For real data, we will use small subsets of the BNU2 and SWU4 datasets. MCFLIRT performed well on these brains, as it was able to effectively remove a lot of the unwanted motion.

Since it is not viable nor helpful to screenshot different timesteps of the motion-corrected brains, the reader should view the corrected brains on their own using FSLview. An example will be presented to Jovo during the deliverable meeting.